**Pre-signed url: overview**

_"Pre-signed URLs allow you to give your users access to a specific object
in your bucket without requiring them to have AWS security credentials or permissions."_

There are two types of pre-signed urls that can be generated:
1. Pre-sign url (limited functionality)
2. Post pre-sign url (extended functionality)

For generating pre-signed and post pre-signed urls with Python,
we will use Boto3 package. You can install it simply with pip by running:

In [ ]:
pip install boto3

For further documentation on boto3, go to: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.generate_presigned_post

**Generate pre-signed URL**

In order to generate pre-signed URL, firstly import needed libraries:

In [ ]:
import boto3
from botocore.client import Config
from botocore.exceptions import ClientError
import environ
import os

Let's implement function for creating pre-signed URL. Notice the usage of boto3 and it's client. This function return
 simple string with generated url.

In [ ]:
def create_get_presigned_url(bucket_name, object_name):
    """Generate a presigned URL S3 POST request to upload a file

    :param bucket_name: string
    :param object_name: string
    :return: Dictionary with the following keys:
        url: URL to post to
        fields: Dictionary of form fields and values to submit with the POST
    :return: None if error.
    """
    # Generate a pre-signed S3 POST URL
    s3_client = boto3.client('s3', config=Config(signature_version='s3v4'), region_name="eu-central-1")
    try:
        response = s3_client.generate_presigned_url('put_object',
                                                    Params={'Bucket': bucket_name,
                                                            'Key': object_name,
                                                            'ContentType': "application/json",
                                                            },
                                                    ExpiresIn=3600)
    except ClientError as e:
        print(str(e))
        return None

    return response

Now, after setting env variables, specifying filename and bucket, pre-signed url can be generated with the function
implemented above.

In [ ]:
env = environ.Env()
os.environ["AWS_ACCESS_KEY_ID"] = env("AWS_ACCESS_KEY_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = env("AWS_SECRET_ACCESS_KEY")
filename = "my_file.parquet"
bucket_name = env("BUCKET")

url = create_get_presigned_url(bucket_name, filename)

Pre-signed URL that we generated, can be used now as callback_url.

Look at the basic_ex.py for instruction of how to initialize client and get algorithms
that client has permission to run.

In order to run algorithm with pre-signed URL,
simply add generated url value to **callback_url** arg.

Example:

In [ ]:
results = algorithm.run(data, evaluate=False, encrypt=True, callback_url=generated_url)

**Post pre-signed URL**

For extended functionality, post pre-signed url can be generated.

Just as before, after Boto3 installation, import required libraries:

In [ ]:
import boto3
from botocore.client import Config
from botocore.exceptions import ClientError
import environ
import os

Now, let's implement a **function for generating post pre-signed url**. Notice, that this function, returns
dictionary with not only generated url, but also *fields* dictionary.
We will need it later when running an algorithm.

In [ ]:
def create_presigned_post(bucket_name, object_name, fields=None, conditions=None, expiration=3600):
    # Generate a presigned S3 POST URL
    s3_client = boto3.client('s3', config=Config(signature_version='s3v4'), region_name="eu-central-1")
    try:
        response = s3_client.generate_presigned_post(
            Bucket=bucket_name,
            Key=object_name,
            Fields=fields,
            Conditions=conditions,
            ExpiresIn=expiration
        )
    except ClientError as e:
        print(str(e))
        return None

    pre_signed_url = response['url']
    fields = response['fields']

    return pre_signed_url, fields

Now, we can use our generated *pre_signed_url* and *fields* as callback_url and callback_param arguments when running
algorithms. In case of post pre-signed url, value for callback_param is required.

Take a look at the basic_ex.py for instruction on how to initialize client and get algorithms
that client has permissions to run.

Let's use *fields* to create callback_param dictionary. We have to assign *fields* to *s3_fields* key.

In [ ]:
callback_param = {'s3_fields': fields}

Now we can run algorithm with callback_url as our post pre-signed url and callback_param as dictionary created above.

In [ ]:
results = algorithm.run(data, evaluate=False, encrypt=True, callback_url=generated_url, callback_param=callback_param)
